# 使用SST辨識文章情緒
The Stanford Sentiment Treebank(SST)

In [1]:
import tensorflow as tf

In [2]:
tf.config.list_physical_devices()

2022-09-20 11:24:07.373588: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:f3:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-09-20 11:24:07.416222: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:f3:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-09-20 11:24:07.416744: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:f3:00.0/numa_node
Your kernel may have been built without NUMA support.


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
tf.test.is_built_with_cuda()

True

## 下載SST資料

In [4]:
import os
data_dir = tf.keras.utils.get_file(
      fname='SST-2.zip',
      origin='https://dl.fbaipublicfiles.com/glue/data/SST-2.zip',
      extract=True)
data_dir = os.path.join(os.path.dirname(data_dir), 'SST-2')

In [5]:
import pandas as pd
trainDF = pd.read_csv(os.path.join(data_dir, 'train.tsv'), sep='\t')
testDF = pd.read_csv(os.path.join(data_dir, 'test.tsv'), sep='\t')

In [6]:
import numpy as np
def createTFDataset(x, y):
  tfds = tf.data.Dataset.from_tensor_slices((x,y))
  return tfds.shuffle(buffer_size=len(x)).batch(512).prefetch(tf.data.AUTOTUNE)

In [7]:
trainDS = createTFDataset(trainDF['sentence'],trainDF['label'])
testDS = tf.data.Dataset.from_tensor_slices((testDF['sentence'])).shuffle(buffer_size=len(testDF['sentence'])).batch(512).prefetch(tf.data.AUTOTUNE)

2022-09-20 11:24:07.978314: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-20 11:24:07.979380: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:f3:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-09-20 11:24:07.979803: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:f3:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-09-20 11:24:07.980111: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:f3:00.0/numa_node
Your kernel

In [8]:
trainDS.element_spec

(TensorSpec(shape=(None,), dtype=tf.string, name=None),
 TensorSpec(shape=(None,), dtype=tf.int64, name=None))

In [9]:
testDS.element_spec

TensorSpec(shape=(None,), dtype=tf.string, name=None)

In [10]:
for example, label in trainDS.take(1):
  print('text: ', example.numpy())
  print('label: ', label.numpy())

text:  [b'before it collapses into exactly the kind of buddy cop comedy '
 b'provide its keenest pleasures to those familiar with bombay musicals '
 b'a fairy tale ' b'this ill-conceived and expensive project '
 b'it does a bang-up job of pleasing the crowds ' b'auto-critique '
 b'show a remarkable ability to document both sides of this emotional car-wreck '
 b'the metaphors are provocative , but too often , the viewer is left puzzled by the mechanics of the delivery . '
 b'the disjointed feel of a bunch of strung-together tv episodes . '
 b'magic and whimsy for children , a heartfelt romance for teenagers and a compelling argument about death , '
 b'had all its vital essence scooped out and '
 b'if you want a movie time trip , the 1960 version is a far smoother ride . '
 b'staged violence overshadows everything , '
 b"it will just as likely make you weep , and it will do so in a way that does n't make you feel like a sucker . "
 b'ray liotta '
 b"several uninteresting , unlikeable peo

In [11]:
VOCAB_SIZE = 1000
encoder = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(trainDS.map(lambda text, label: text))

2022-09-20 11:24:09.069543: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


In [12]:
vocab = np.array(encoder.get_vocabulary())
vocab[:20]

array(['', '[UNK]', 'the', 'a', 'and', 'of', 'to', 's', 'is', 'that',
       'in', 'it', 'as', 'with', 'an', 'film', 'its', 'for', 'movie',
       'this'], dtype='<U14')

In [13]:
encoded_example = encoder(example)[:3].numpy()
encoded_example

array([[192,  11,   1,  44, 870,   2, 149,   5,   1,   1,  53,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0],
       [  1,  16,   1,   1,   6, 151, 372,  13,   1,   1,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0],
       [  3,   1, 170,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0]])

In [14]:
for n in range(3):
  print("Original: ", example[n].numpy())
  print("Round-trip: ", " ".join(vocab[encoded_example[n]]))
  print()

Original:  b'before it collapses into exactly the kind of buddy cop comedy '
Round-trip:  before it [UNK] into exactly the kind of [UNK] [UNK] comedy                                  

Original:  b'provide its keenest pleasures to those familiar with bombay musicals '
Round-trip:  [UNK] its [UNK] [UNK] to those familiar with [UNK] [UNK]                                   

Original:  b'a fairy tale '
Round-trip:  a [UNK] tale                                          



<img src="https://www.tensorflow.org/static/text/tutorials/images/bidirectional.png">

In [15]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)
])

In [16]:
print([layer.supports_masking for layer in model.layers])

[False, True, True, True, True]


In [17]:
# predict on a sample text without padding.

sample_text = ('The movie was cool. The animation and the graphics '
               'were out of this world. I would recommend this movie.')
predictions = model.predict(np.array([sample_text]))
print(predictions[0])

2022-09-20 11:24:13.129237: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8401
2022-09-20 11:24:13.269724: I tensorflow/stream_executor/cuda/cuda_blas.cc:1760] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


[0.49644384]


In [18]:
# predict on a sample text with padding

padding = "the " * 2000
predictions = model.predict(np.array([sample_text, padding]))
print(predictions[0])

[0.49644384]


In [19]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [20]:
history = model.fit(trainDS, epochs=10)

Epoch 1/10
132/132 [==============================] - 7s 31ms/step - loss: 0.6859 - accuracy: 0.5567
Epoch 2/10
132/132 [==============================] - 4s 30ms/step - loss: 0.6006 - accuracy: 0.6920
Epoch 3/10
132/132 [==============================] - 4s 29ms/step - loss: 0.4996 - accuracy: 0.7586
Epoch 4/10
132/132 [==============================] - 4s 31ms/step - loss: 0.4638 - accuracy: 0.7704
Epoch 5/10
132/132 [==============================] - 4s 29ms/step - loss: 0.4491 - accuracy: 0.7736
Epoch 6/10
132/132 [==============================] - 4s 29ms/step - loss: 0.4415 - accuracy: 0.7753
Epoch 7/10
132/132 [==============================] - 4s 28ms/step - loss: 0.4362 - accuracy: 0.7771
Epoch 8/10
132/132 [==============================] - 4s 29ms/step - loss: 0.4324 - accuracy: 0.7787
Epoch 9/10
132/132 [==============================] - 4s 29ms/step - loss: 0.4297 - accuracy: 0.7802
Epoch 10/10
132/132 [==============================] - 4s 29ms/step - loss: 0.4271 - accura

In [23]:
pred = model.predict(testDS)
pred

array([[0.96720606],
       [0.08715189],
       [0.9040408 ],
       ...,
       [0.96857655],
       [0.78351796],
       [0.501945  ]], dtype=float32)

In [32]:
for i, j in zip(testDS.unbatch(), pred):
    print(i, np.round(j))

tf.Tensor(b'a film so tedious that it is impossible to care whether that boast is true or not .', shape=(), dtype=string) [1.]
tf.Tensor(b'those who would follow haneke on his creepy explorations ... are rewarded by brutal , committed performances from huppert and magimel .', shape=(), dtype=string) [0.]
tf.Tensor(b'windtalkers is shapelessly gratifying , the kind of movie that invites you to pick apart its faults even as you have to admit that somehow it hit you where you live .', shape=(), dtype=string) [1.]
tf.Tensor(b'far from perfect , but its heart is in the right place ... innocent and well-meaning .', shape=(), dtype=string) [0.]
tf.Tensor(b'bad beyond belief and ridiculous beyond description .', shape=(), dtype=string) [0.]
tf.Tensor(b'as written by michael berg and michael j. wilson from a story by wilson , this relentless , all-wise-guys-all-the-time approach tries way too hard and gets tiring in no time at all .', shape=(), dtype=string) [1.]
tf.Tensor(b'scores a few points

In [25]:
np.round(pred)

array([[1.],
       [0.],
       [1.],
       ...,
       [1.],
       [1.],
       [1.]], dtype=float32)